In [1]:
from tinygrad.densetensor import DenseTensor, cl_queue
from tinygrad.sparsetensor import SparseTensor
import numpy as np
import tinygrad.optim as optim
from extra.utils import get_parameters


%load_ext autoreload
%autoreload 2

DEVICE:GPU


In [2]:
dim1 = 4
dim2 = 4
dim3 = 4

np.random.seed(9)

In [3]:
x_init = np.random.randn(dim1,dim2).astype(np.float32)
w_init = np.random.randn(dim2,dim3).astype(np.float32)
x = DenseTensor(x_init)
y = DenseTensor(np.random.randn(dim1,dim3))

warning, (4, 4) isn't float32, it's float64


W_TEST = SparseTensor.uniform(dim2,dim3, randsparsity=0.6).to_numpy()
W_TEST

In [4]:
W_TEST = SparseTensor(w_init).to_numpy()
W_TEST

array([[ 1.54272962, -0.90072119, -0.13712502,  1.29757905],
       [ 0.67527115,  0.03195812,  0.9181459 ,  0.38050947],
       [ 0.5163675 , -0.35523945,  0.208777  ,  0.32841107],
       [-0.49822477, -2.09177685, -0.08258774,  2.45518255]])

In [5]:
class MLP:
  def __init__(self, init_dense):
    #self.W = DenseTensor(init_dense)
    self.W = DenseTensor(W_TEST)

  def parameters(self):
    return get_parameters(self)

  def forward(self, x):
    out = x.dot(self.W)
    out = out.softmax()
    #out = out.mul(self.m).add(self.m).sum()
    return out

In [6]:
class MLP2:
  def __init__(self):
    self.W = SparseTensor(W_TEST)
    #self.W = SparseTensor.uniform(dim2,dim3)
    #self.W = SparseTensor.uniform(dim2,dim3)

  def parameters(self):
    return get_parameters(self)

  def forward(self, x):
    out = x.dot(self.W)
    out = out.softmax()
    #out = out.mul(self.m).add(self.m).sum()
    return out

In [7]:
def loss_fn(y, y_pred):
    return ((y-y_pred)**2)**.5

In [8]:
model2 = MLP2()

In [9]:
model2.W.get_nnzs()

array([4, 4, 4, 4], dtype=uint32)

In [10]:
dense_init = model2.W.to_numpy()
dense_init

array([[ 1.54272962, -0.90072119, -0.13712502,  1.29757905],
       [ 0.67527115,  0.03195812,  0.9181459 ,  0.38050947],
       [ 0.5163675 , -0.35523945,  0.208777  ,  0.32841107],
       [-0.49822477, -2.09177685, -0.08258774,  2.45518255]])

In [11]:
model = MLP(dense_init)

### Dense

In [12]:
iters = 4
LR = 0.1
optimizer = optim.SGD(model.parameters(), lr=LR)

In [13]:
for i in range(iters):
    res = model.forward(x)
    optimizer.zero_grad()
    loss = loss_fn(res, y)
    loss.backward()
    optimizer.step()

In [14]:
model.parameters()

[<DenseTensor <GPUBuffer with shape (4, 4)> with grad <GPUBuffer with shape (4, 4)>>,
 <DenseTensor <GPUBuffer with shape (4, 4)> with grad None>]

In [15]:
res.cpu().data

array([[1.5482739e-01, 4.4537634e-01, 1.5450318e-01, 2.4529316e-01],
       [3.7580851e-02, 8.8807660e-01, 5.4560788e-02, 1.9781798e-02],
       [5.5333241e-03, 5.7795382e-04, 1.4139037e-02, 9.7974968e-01],
       [3.1354329e-01, 1.2243453e-02, 2.5163570e-01, 4.2257753e-01]],
      dtype=float32)

In [16]:
loss.cpu().data

array([[2.8269374 , 1.3586556 , 0.38181752, 0.02402224],
       [1.0928804 , 0.1543209 , 1.2492497 , 1.3696189 ],
       [0.66198593, 0.05683525, 0.51404166, 0.5433303 ],
       [0.6893563 , 0.935305  , 1.6656145 , 0.5728804 ]], dtype=float32)

In [17]:
res.grad.cpu().data

array([[ 0.99999994,  1.        ,  1.        , -1.        ],
       [-0.99999994, -1.        , -1.        , -0.99999994],
       [ 0.99999994,  1.0000001 ,  0.9999998 ,  1.        ],
       [ 1.        ,  0.99999994, -0.99999994,  1.        ]],
      dtype=float32)

In [18]:
model.W.grad.cpu().data

array([[ 4.68998738e-02,  2.07030587e-03, -1.11655496e-01,
         6.26853183e-02],
       [ 8.96495506e-02, -5.89045994e-02, -2.88413972e-01,
         2.57669002e-01],
       [ 2.02863380e-01, -2.32623696e-01, -7.71122336e-01,
         8.00882697e-01],
       [ 6.69956878e-02, -1.60518990e-04, -1.63217157e-01,
         9.63819921e-02]], dtype=float32)

### Second

In [19]:
w_init

array([[ 1.5427296 , -0.9007212 , -0.13712502,  1.297579  ],
       [ 0.67527115,  0.03195812,  0.9181459 ,  0.38050947],
       [ 0.5163675 , -0.35523945,  0.208777  ,  0.32841107],
       [-0.49822477, -2.0917768 , -0.08258774,  2.4551826 ]],
      dtype=float32)

In [20]:
model2 = MLP2()

In [21]:
optimizer2 = optim.SGD(model2.parameters(), lr=LR)

In [22]:
for i in range(iters):
    res2 = model2.forward(x)
    optimizer2.zero_grad()
    loss2 = loss_fn(res2, y)
    loss2.backward()
    optimizer2.step()

#### fwd

In [23]:
res2.cpu().data==res.cpu().data

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [24]:
res2.cpu().data

array([[1.5482739e-01, 4.4537634e-01, 1.5450318e-01, 2.4529316e-01],
       [3.7580851e-02, 8.8807660e-01, 5.4560788e-02, 1.9781798e-02],
       [5.5333241e-03, 5.7795382e-04, 1.4139037e-02, 9.7974968e-01],
       [3.1354329e-01, 1.2243453e-02, 2.5163570e-01, 4.2257753e-01]],
      dtype=float32)

In [25]:
res.cpu().data - res2.cpu().data

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

#### loss

In [26]:
loss.cpu().data==loss2.cpu().data

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [27]:
loss2.cpu().data

array([[2.8269374 , 1.3586556 , 0.38181752, 0.02402224],
       [1.0928804 , 0.1543209 , 1.2492497 , 1.3696189 ],
       [0.66198593, 0.05683525, 0.51404166, 0.5433303 ],
       [0.6893563 , 0.935305  , 1.6656145 , 0.5728804 ]], dtype=float32)

In [28]:
loss.cpu().data - loss2.cpu().data

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

#### grad

In [29]:
res.grad.cpu().data==res2.grad.cpu().data

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [30]:
res2.grad.cpu().data

array([[ 0.99999994,  1.        ,  1.        , -1.        ],
       [-0.99999994, -1.        , -1.        , -0.99999994],
       [ 0.99999994,  1.0000001 ,  0.9999998 ,  1.        ],
       [ 1.        ,  0.99999994, -0.99999994,  1.        ]],
      dtype=float32)

In [31]:
res.grad.cpu().data-res2.grad.cpu().data

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

#### weight

In [32]:
def to_dense(data, cols, nnzs, ellw, shape):
    out = np.zeros(shape)
    for row in range(shape[0]):
        for icol in range(nnzs[row]):
            #print('idx:',row,cols[row*ellw+icol])
            out[row,cols[row*ellw+icol]] = data[row*ellw+icol]
    return out

In [33]:
model.W.grad, model2.W.grad

(<DenseTensor <GPUBuffer with shape (4, 4)> with grad None>,
 <SparseTensor <GPUBuffer with shape (16,)> with grad None>)

In [34]:
model.W.grad.cpu().data

array([[ 4.68998738e-02,  2.07030587e-03, -1.11655496e-01,
         6.26853183e-02],
       [ 8.96495506e-02, -5.89045994e-02, -2.88413972e-01,
         2.57669002e-01],
       [ 2.02863380e-01, -2.32623696e-01, -7.71122336e-01,
         8.00882697e-01],
       [ 6.69956878e-02, -1.60518990e-04, -1.63217157e-01,
         9.63819921e-02]], dtype=float32)

In [35]:
model2.W.grad.to_numpy()

array([[ 4.68998738e-02,  2.07030587e-03, -1.11655496e-01,
         6.26853183e-02],
       [ 8.96495506e-02, -5.89045994e-02, -2.88413972e-01,
         2.57669002e-01],
       [ 2.02863380e-01, -2.32623696e-01, -7.71122336e-01,
         8.00882697e-01],
       [ 6.69956878e-02, -1.60518990e-04, -1.63217157e-01,
         9.63819921e-02]])

In [36]:
model.W.grad.cpu().data == model2.W.grad.to_numpy()

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [37]:
model.W.cpu().data

array([[ 1.5286762 , -0.9013099 , -0.10056853,  1.2756649 ],
       [ 0.6493594 ,  0.05374243,  1.0139954 ,  0.28878734],
       [ 0.4592371 , -0.26974407,  0.46670884,  0.04211424],
       [-0.5182528 , -2.0914817 , -0.02907925,  2.4214065 ]],
      dtype=float32)

In [38]:
model2.W.to_numpy()

array([[ 1.52867615, -0.90130991, -0.10056853,  1.27566493],
       [ 0.64935941,  0.05374243,  1.01399541,  0.28878734],
       [ 0.4592371 , -0.26974407,  0.46670884,  0.04211424],
       [-0.51825279, -2.09148169, -0.02907925,  2.42140651]])

In [39]:
model.W.cpu().data == model2.W.to_numpy()

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [40]:
cols

NameError: name 'cols' is not defined

In [ ]:
nnzs

In [ ]:
denserec = model.W.cpu().data 
denserec

In [ ]:
sparserec = model2.W.to_numpy()
sparserec

In [ ]:
denserec - sparserec

In [ ]:
sparserec2 = model2.W.to_numpy(dual=True)
sparserec2.T

In [ ]:
denserec - sparserec2.T

In [ ]:
seedmat - denserec